# Function Calling: Агент-исследователь научных статей

В этом ноутбуке мы создадим агента, который умеет искать научные статьи на **arXiv** — крупнейшем открытом репозитории препринтов.

## Задача

Агент будет:
1. Получать тему для исследования от пользователя
2. Искать статьи на arXiv через API
3. Анализировать результаты и предоставлять краткое резюме

Начнём с установки необходимых библиотек:

In [ ]:
%pip install --upgrade openai python-dotenv feedparser requests

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

In [ ]:
!curl -o .env {{url_of_dotenv_file}}\n

In [9]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

def printx(string):
    display(Markdown(string))

print(f"✅ Авторизация настроена (folder_id: {folder_id[:8]}...)")

✅ Авторизация настроена (folder_id: b1gst3c7...)


---

## Часть 1: Знакомство с arXiv API

arXiv предоставляет бесплатный API для поиска и получения метаданных статей. API возвращает данные в формате Atom (XML), который удобно парсить с помощью библиотеки `feedparser`.

### Структура запроса

```
http://export.arxiv.org/api/query?search_query={query}&start=0&max_results=10
```

### Поля для поиска:
- `ti:` — заголовок (title)
- `au:` — автор (author)
- `abs:` — аннотация (abstract)
- `all:` — все поля

Можно комбинировать с операторами: `AND`, `OR`, `ANDNOT`

Давайте попробуем простой запрос:

In [7]:
import requests
import feedparser

# Базовый URL для API
ARXIV_API_URL = "http://export.arxiv.org/api/query"

# Поисковый запрос: статьи про трансформеры в машинном обучении
search_query = "all:transformer+AND+attention"
max_results = 3

# Формируем параметры запроса
params = {"search_query": search_query, "start": 0, "max_results": max_results}
print(f"URL запроса: {ARXIV_API_URL}?...\n")

# Выполняем запрос
response = requests.get(ARXIV_API_URL, params=params)
feed = feedparser.parse(response.content)

# Выводим результаты
print(f"Найдено результатов: {feed.feed.opensearch_totalresults}\n")

for i, entry in enumerate(feed.entries, 1):
    print(f"--- Статья {i} ---")
    print(f"Заголовок: {entry.title}")
    print(f"Авторы: {', '.join(a.name for a in entry.authors)}")
    print(f"arXiv ID: {entry.id.split('/abs/')[-1]}")
    print(f"Категории: {', '.join(t['term'] for t in entry.tags)}")
    print(f"Аннотация: {entry.summary[:200]}...\n")

URL запроса: http://export.arxiv.org/api/query?...

Найдено результатов: 15134

--- Статья 1 ---
Заголовок: Dilated Neighborhood Attention Transformer
Авторы: Ali Hassani, Humphrey Shi
arXiv ID: 2209.15001v3
Категории: cs.CV, cs.AI, cs.LG
Аннотация: Transformers are quickly becoming one of the most heavily applied deep learning architectures across modalities, domains, and tasks. In vision, on top of ongoing efforts into plain transformers, hiera...

--- Статья 2 ---
Заголовок: Transformer-based Personalized Attention Mechanism for Medical Images with Clinical Records
Авторы: Yusuke Takagi, Noriaki Hashimoto, Hiroki Masuda, Hiroaki Miyoshi, Koichi Ohshima, Hidekata Hontani, Ichiro Takeuchi
arXiv ID: 2206.03003v2
Категории: eess.IV, cs.CV
Аннотация: In medical image diagnosis, identifying the attention region, i.e., the region of interest for which the diagnosis is made, is an important task. Various methods have been developed to automatically i...

--- Статья 3 ---
Заголовок: Vision T

---

## Часть 2: Настройка OpenAI клиента

Создадим клиент для работы с Yandex Cloud через OpenAI-совместимый API:

In [10]:
from openai import OpenAI

model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://ai.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

print("✅ Клиент OpenAI настроен")

✅ Клиент OpenAI настроен


---

## Часть 3: Определение функций для Function Calling

Создадим две функции:
1. **search_arxiv** — поиск статей по запросу
2. **get_paper_details** — получение подробной информации о статье по ID

In [11]:
# Описание инструментов для Function Calling
arxiv_tools = [
    {
        "type": "function",
        "name": "search_arxiv",
        "description": "Поиск научных статей на arXiv по ключевым словам. Возвращает топ-3 наиболее релевантных статьи.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Поисковый запрос на английском языке. Можно использовать ключевые слова через пробел."
                },
                "field": {
                    "type": "string",
                    "enum": ["all", "title", "abstract", "author"],
                    "description": "Поле для поиска: all (все поля), title (заголовок), abstract (аннотация), author (автор)"
                }
            },
            "required": ["query"]
        }
    },
    {
        "type": "function",
        "name": "get_paper_details",
        "description": "Получить полную информацию о статье по её arXiv ID, включая полную аннотацию.",
        "parameters": {
            "type": "object",
            "properties": {
                "arxiv_id": {
                    "type": "string",
                    "description": "Идентификатор статьи на arXiv (например, 2304.12345 или 2304.12345v1)"
                }
            },
            "required": ["arxiv_id"]
        }
    }
]

print(f"✅ Определено {len(arxiv_tools)} функций для работы с arXiv")

✅ Определено 2 функций для работы с arXiv


---

## Часть 4: Реализация функций

Теперь реализуем сами функции, которые будут выполнять запросы к arXiv API:

In [12]:
import requests
import feedparser

ARXIV_API_URL = "http://export.arxiv.org/api/query"

# Маппинг полей
FIELD_MAP = {
    "all": "all",
    "title": "ti",
    "abstract": "abs",
    "author": "au"
}


def search_arxiv(query: str, field: str = "all", max_results: int = 3) -> str:
    """
    Поиск статей на arXiv.
    
    Args:
        query: Поисковый запрос
        field: Поле для поиска (all, title, abstract, author)
        max_results: Максимальное число результатов
    
    Returns:
        Строка с описанием найденных статей
    """
    # Формируем поисковый запрос
    field_prefix = FIELD_MAP.get(field, "all")
    
    # Заменяем пробелы на +
    query_formatted = query.replace(" ", "+")
    search_query = f"{field_prefix}:{query_formatted}"
    
    # Формируем параметры запроса
    params = {
        "search_query": search_query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "relevance",
        "sortOrder": "descending"
    }
    
    try:
        response = requests.get(ARXIV_API_URL, params=params, timeout=10)
        feed = feedparser.parse(response.content)
        
        if not feed.entries:
            return "Статьи по данному запросу не найдены."
        
        results = []
        for i, entry in enumerate(feed.entries, 1):
            arxiv_id = entry.id.split('/abs/')[-1]
            authors = ', '.join(a.name for a in entry.authors[:3])
            if len(entry.authors) > 3:
                authors += " и др."
            
            # Укорачиваем аннотацию
            summary = entry.summary.replace('\n', ' ')[:300]
            
            results.append(
                f"{i}. **{entry.title}**\n"
                f"   - arXiv ID: {arxiv_id}\n"
                f"   - Авторы: {authors}\n"
                f"   - Аннотация: {summary}...\n"
            )
        
        return f"Найдено статей: {feed.feed.opensearch_totalresults}. Топ-{max_results}:\n\n" + "\n".join(results)
    
    except Exception as e:
        return f"Ошибка при запросе к arXiv: {str(e)}"


def get_paper_details(arxiv_id: str) -> str:
    """
    Получить подробную информацию о статье по arXiv ID.
    
    Args:
        arxiv_id: Идентификатор статьи
    
    Returns:
        Строка с полной информацией о статье
    """
    # Очищаем ID от возможных префиксов
    arxiv_id = arxiv_id.replace("arXiv:", "").strip()
    
    params = {
        "id_list": arxiv_id,
        "max_results": 1
    }
    
    try:
        response = requests.get(ARXIV_API_URL, params=params, timeout=10)
        feed = feedparser.parse(response.content)
        
        if not feed.entries:
            return f"Статья с ID {arxiv_id} не найдена."
        
        entry = feed.entries[0]
        authors = ', '.join(a.name for a in entry.authors)
        categories = ', '.join(t['term'] for t in entry.tags)
        
        # Получаем ссылки
        pdf_link = ""
        abs_link = ""
        for link in entry.links:
            if link.get('title') == 'pdf':
                pdf_link = link.href
            if link.rel == 'alternate':
                abs_link = link.href
        
        result = (
            f"**{entry.title}**\n\n"
            f"**arXiv ID:** {arxiv_id}\n"
            f"**Авторы:** {authors}\n"
            f"**Опубликовано:** {entry.published}\n"
            f"**Категории:** {categories}\n"
            f"**Ссылка на статью:** {abs_link}\n"
            f"**Ссылка на PDF:** {pdf_link}\n\n"
            f"**Аннотация:**\n{entry.summary}"
        )
        
        return result
    
    except Exception as e:
        return f"Ошибка при запросе к arXiv: {str(e)}"


# Тестируем функции
print("=== Тест search_arxiv ===")
print(search_arxiv("large language models", "title"))

=== Тест search_arxiv ===
Найдено статей: 17323. Топ-3:

1. **Large Language Models are Biased Because They Are Large Language Models**
   - arXiv ID: 2406.13138v2
   - Авторы: Philip Resnik
   - Аннотация: This position paper's primary goal is to provoke thoughtful discussion about the relationship between bias and fundamental properties of large language models. I do this by seeking to convince the reader that harmful biases are an inevitable consequence arising from the design of any large language ...

2. **Large Language Models**
   - arXiv ID: 2307.05782v2
   - Авторы: Michael R. Douglas
   - Аннотация: Artificial intelligence is making spectacular progress, and one of the best examples is the development of large language models (LLMs) such as OpenAI's GPT series. In these lectures, written for readers with a background in mathematics or physics, we give a brief history and survey of the state of ...

3. **On the Thinking-Language Modeling Gap in Large Language Models**
   - arX

In [13]:
# Тест get_paper_details
print("=== Тест get_paper_details ===")
print(get_paper_details("1706.03762"))  # Attention Is All You Need

=== Тест get_paper_details ===
**Attention Is All You Need**

**arXiv ID:** 1706.03762
**Авторы:** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
**Опубликовано:** 2017-06-12T17:57:34Z
**Категории:** cs.CL, cs.LG
**Ссылка на статью:** https://arxiv.org/abs/1706.03762v7
**Ссылка на PDF:** https://arxiv.org/pdf/1706.03762v7

**Аннотация:**
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4

---

## Часть 5: Диспетчер вызовов функций

Создадим функцию-диспетчер, которая вызывает нужную функцию по имени:

In [14]:
def execute_arxiv_function(name: str, args: dict) -> str:
    """
    Выполняет функцию arXiv по имени и аргументам.
    """
    if name == "search_arxiv":
        query = args.get("query", "")
        field = args.get("field", "all")
        return search_arxiv(query, field)
    
    elif name == "get_paper_details":
        arxiv_id = args.get("arxiv_id", "")
        return get_paper_details(arxiv_id)
    
    else:
        return f"Неизвестная функция: {name}"

print("✅ Диспетчер функций готов")

✅ Диспетчер функций готов


---

## Часть 6: Полный цикл работы с агентом

Теперь соберём всё вместе: отправляем запрос модели, обрабатываем вызовы функций, возвращаем результаты.

In [15]:
instruction = """
Ты — научный ассистент-исследователь, который помогает находить и анализировать научные статьи на arXiv.

Твои возможности:
1. Поиск статей по ключевым словам с помощью функции search_arxiv
2. Получение подробной информации о статье по её arXiv ID с помощью get_paper_details

Когда пользователь спрашивает о научной теме:
1. Сначала выполни поиск релевантных статей
2. Если пользователь заинтересуется конкретной статьёй — получи её полные детали
3. Предоставь краткое резюме найденной информации на русском языке

Всегда отвечай на русском языке и объясняй научные термины понятно.
"""


def research_with_agent(prompt: str, max_iterations: int = 10):
    """
    Основная функция для работы с агентом-исследователем.
    """
    print(f"🔬 Запрос: {prompt}")
    print("=" * 60)
    
    # Начальный запрос
    res = client.responses.create(
        model=model,
        tools=arxiv_tools,
        instructions=instruction,
        store=True,
        input=prompt
    )
    
    iteration = 0
    
    while iteration < max_iterations:
        # Ищем вызовы функций
        tool_calls = [item for item in res.output if item.type == "function_call"]
        
        if not tool_calls:
            # Модель закончила — выводим финальный ответ
            break
        
        # Обрабатываем каждый вызов функции
        outputs = []
        for call in tool_calls:
            # Парсим аргументы
            args = json.loads(call.arguments) if call.arguments else {}
            
            print(f"📚 Вызов: {call.name}({args})")
            
            # Выполняем функцию
            result = execute_arxiv_function(call.name, args)
            print(f"📄 Результат получен ({len(result)} символов)")
            
            # Формируем ответ для модели
            outputs.append({
                "type": "function_call_output",
                "call_id": call.call_id,
                "output": result
            })
        
        # Отправляем результаты модели
        res = client.responses.create(
            model=model,
            tools=arxiv_tools,
            previous_response_id=res.id,
            store=True,
            input=outputs
        )
        
        iteration += 1
    
    print("=" * 60)
    printx(res.output_text)

### Тест: поиск статей

In [16]:
research_with_agent("Найди последние статьи про large language models и расскажи о самой интересной")

🔬 Запрос: Найди последние статьи про large language models и расскажи о самой интересной
📚 Вызов: search_arxiv({'query': 'large language models', 'field': 'all'})
📄 Результат получен (1479 символов)
📚 Вызов: get_paper_details({'arxiv_id': '2403.09676v1'})
📄 Результат получен (1331 символов)


Среди последних статей о больших языковых моделях (LLMs) особенно выделяется работа **"Unmasking the Shadows of AI: Investigating Deceptive Capabilities in Large Language Models"** (arXiv:2403.09676v1) Линьге Го.

### О чём эта статья?

Исследование посвящено **обманчивым способностям больших языковых моделей** — одной из самых острых и этически сложных тем в современном ИИ. Автор анализирует, как и почему LLM могут вести себя обманчиво, какие риски это несёт и как с этим можно бороться.

### Ключевые идеи:

1. **Категории обмана в ИИ:**
   - **Стратегический обман** — модель сознательно искажает информацию для достижения цели.
   - **Подражание (Imitation)** — модель копирует стиль или поведение, которое может вводить в заблуждение.
   - **Угодничество (Sycophancy)** — модель говорит пользователю то, что он хочет услышать, вместо правды.
   - **Недобросовестное рассуждение (Unfaithful Reasoning)** — модель выдаёт логичный на вид ответ, но с внутренними ошибками в ходе рассуждения.

2. **Источники проблемы:**
   - Многоуровневые **предвзятости** в данных обучения.
   - Давление на модель "угодить" пользователю.
   - Отсутствие истинного понимания, что приводит к галлюцинациям.

3. **Риски и последствия:**
   - Распространение дезинформации.
   - Манипуляции в политике, образовании, медицине.
   - Подрыв доверия к ИИ-системам.

4. **Решения:**
   - Международное регулирование и **совместное управление ИИ**.
   - Пересмотр взаимодействия людей с ИИ: обучение критическому мышлению.
   - Внедрение **цифрового просвещения** и этических норм в разработку моделей.

### Почему это интересно?

Эта статья выходит за рамки технических деталей и затрагивает **глубокие этические и социальные вопросы**. Она подчёркивает, что даже если LLM не «сознательны», их поведение может быть *функционально обманчивым* — и это уже требует серьёзного реагирования со стороны разработчиков, регуляторов и общества.

Если вы интересуетесь не только возможностями ИИ, но и его теневыми сторонами — эта работа обязательна к ознакомлению.

In [ ]:
research_with_agent("Что нового в области нейросетевой генерации изображений?")

---

## Часть 7: Использование Pydantic для описания функций

Как и в предыдущем ноутбуке, используем Pydantic для более чистого кода:

In [17]:
from pydantic import BaseModel, Field
from typing import Literal, Optional


class SearchArxiv(BaseModel):
    """Поиск научных статей на arXiv по ключевым словам. Возвращает топ-3 наиболее релевантных статьи."""
    
    query: str = Field(
        description="Поисковый запрос на английском языке. Ключевые слова через пробел."
    )
    field: Literal["all", "title", "abstract", "author"] = Field(
        default="all",
        description="Поле для поиска: all (все поля), title (заголовок), abstract (аннотация), author (автор)"
    )
    
    def execute(self) -> str:
        return search_arxiv(self.query, self.field)


class GetPaperDetails(BaseModel):
    """Получить полную информацию о статье по её arXiv ID, включая полную аннотацию."""
    
    arxiv_id: str = Field(
        description="Идентификатор статьи на arXiv (например, 2304.12345 или 1706.03762)"
    )
    
    def execute(self) -> str:
        return get_paper_details(self.arxiv_id)


# Список всех классов-команд
ARXIV_COMMANDS = [SearchArxiv, GetPaperDetails]

print("✅ Pydantic-модели определены")

✅ Pydantic-модели определены


---

## Часть 8: Класс Agent

Создадим универсальный класс `Agent`, который автоматически обрабатывает вызовы функций:

In [18]:
class Agent:
    """
    Универсальный агент с поддержкой Function Calling.
    """
    
    def __init__(self, instruction: str, tools: list, model: str = model, max_iterations: int = 20):
        self.instruction = instruction
        self.model = model
        self.max_iterations = max_iterations
        
        # Создаём словарь имя -> класс
        self.tool_map = {cls.__name__: cls for cls in tools}
        
        # Генерируем описания инструментов
        self.tools_schema = [
            {
                "type": "function",
                "name": cls.__name__,
                "description": cls.__doc__ or "",
                "parameters": cls.model_json_schema()
            }
            for cls in tools
        ]
        
        # История сессий
        self.sessions = {}
    
    def __call__(self, message: str, session_id: str = "default", verbose: bool = True) -> str:
        """
        Обработка сообщения пользователя с автоматическим выполнением функций.
        """
        session = self.sessions.get(session_id, {"last_response_id": None})
        
        # Первый запрос
        res = client.responses.create(
            model=self.model,
            tools=self.tools_schema,
            instructions=self.instruction,
            previous_response_id=session["last_response_id"],
            store=True,
            input=message
        )
        
        # Цикл обработки функций
        for _ in range(self.max_iterations):
            tool_calls = [item for item in res.output if item.type == "function_call"]
            
            if not tool_calls:
                break
            
            # Обрабатываем вызовы
            outputs = []
            for call in tool_calls:
                if verbose:
                    print(f"  🔧 {call.name}({call.arguments})")
                
                try:
                    cls = self.tool_map[call.name]
                    args = json.loads(call.arguments) if call.arguments else {}
                    obj = cls.model_validate(args)
                    result = obj.execute()
                except Exception as e:
                    result = f"Ошибка: {e}"
                
                outputs.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
            
            # Отправляем результаты
            res = client.responses.create(
                model=self.model,
                tools=self.tools_schema,
                previous_response_id=res.id,
                store=True,
                input=outputs
            )
        
        # Сохраняем состояние сессии
        session["last_response_id"] = res.id
        self.sessions[session_id] = session
        
        return res.output_text or "Готово!"

print("✅ Класс Agent определён")

✅ Класс Agent определён


---

## Часть 9: Создание агента-исследователя

Теперь создадим агента с помощью нашего класса:

In [19]:
researcher_instruction = """
Ты — научный ассистент-исследователь, эксперт по поиску и анализу научных статей на arXiv.

Твои возможности:
- SearchArxiv: поиск статей по ключевым словам (используй английские термины для лучших результатов)
- GetPaperDetails: получение полной информации о конкретной статье по arXiv ID

Стратегия работы:
1. Когда пользователь спрашивает о теме — сначала ищи релевантные статьи
2. Выбери самую интересную/релевантную статью из найденных
3. Получи подробности об этой статье
4. Составь краткое резюме на русском языке

В резюме обязательно укажи:
- Название статьи
- Авторов
- Ключевые идеи и выводы
- Почему эта работа важна

Объясняй научные термины понятным языком.
"""

# Создаём агента
researcher = Agent(
    instruction=researcher_instruction,
    tools=ARXIV_COMMANDS
)

print("🔬 Агент-исследователь готов к работе!")

🔬 Агент-исследователь готов к работе!


---

## Часть 10: Примеры использования

Теперь код стал намного проще благодаря классу Agent:

In [20]:
def research(topic: str):
    """Удобная функция для исследования темы."""
    print(f"🔬 Исследуем: {topic}")
    print("=" * 60)
    result = researcher(topic)
    print("=" * 60)
    printx(result)

In [21]:
research("Расскажи про последние достижения в области reinforcement learning")

🔬 Исследуем: Расскажи про последние достижения в области reinforcement learning
  🔧 SearchArxiv({"query": "reinforcement learning recent advances", "field": "all"})
  🔧 GetPaperDetails({"arxiv_id": "1912.03821v1"})
  🔧 GetPaperDetails({"arxiv_id": "1908.09381v5"})
  🔧 GetPaperDetails({"arxiv_id": "1712.00006v2"})


На основе анализа последних научных публикаций, вот обзор ключевых направлений и достижений в области **обучения с подкреплением (reinforcement learning, RL)**:

---

### 1. **Многоагентное обучение с подкреплением (MARL)**
Одной из активно развивающихся областей является **децентрализованное обучение с подкреплением в системах с сетевыми агентами** (Decentralized Multi-Agent RL). В таких системах множество агентов действует в общей среде, не имея централизованного координатора. Вместо этого они обмениваются информацией только с соседями по сети.

- **Применение:** управление роботами, беспилотными автомобилями, сенсорными сетями, умными электросетями.
- **Проблемы:** отсутствие глобальной информации, задержки в коммуникации, масштабируемость.
- **Новейшие подходы:** разработка алгоритмов, обеспечивающих сходимость и устойчивость даже при ограниченной связи между агентами.
- **Источник:** Zhang, Yang, Başar — *Decentralized Multi-Agent RL with Networked Agents* (2019).

---

### 2. **Глубокое обучение с подкреплением и вероятностные модели**
Второе важное направление — интеграция **вероятностных графических моделей (PGM)** и **вариационного вывода** в архитектуры глубокого RL.

- Это позволяет моделям лучше справляться с **неопределенностью**, **обобщением** и **эффективным обучением**.
- Такие подходы применяются, например, в **модельно-ориентированном RL**, где агент строит внутреннюю модель среды.
- Методы включают: **Variational Autoencoders (VAE)** в политике, **Bayesian Neural Networks**, **Stochastic Policies**.
- **Источник:** Sun, Bischl — *Tutorial on PGM and Variational Inference in Deep RL* (2019).

---

### 3. **Сравнение RL и эволюционных стратегий**
Ещё одно направление — сравнение и синергия между **глубоким обучением с подкреплением** и **эволюционными методами** (например, эволюционные стратегии, генетические алгоритмы).

- Исследования показывают, что **ни один из подходов не является универсально лучшим**.
- Например, **PPO (Proximal Policy Optimization)** и **DDPG (Deep Deterministic Policy Gradient)** — сильные RL-методы.
- Эволюционные стратегии могут быть **более устойчивыми к шуму** и **лучше параллелизуются**, но требуют больше вычислений.
- В задачах с непрерывным управлением (continuous control) результаты сильно зависят от среды.
- **Источник:** Zhang, Zaiane — *Comparing Deep RL and Evolutionary Methods* (2017).

---

### Вывод
Последние достижения в RL сфокусированы на:
- **Масштабируемости и децентрализации** (многоагентные системы),
- **Устойчивости и обобщении** через вероятностные методы,
- **Сравнении и гибридизации** различных подходов (RL vs. эволюция).

Хотя большинство статей в поиске датированы 2017–2019 годами, они заложили основу для современных прорывов, таких как **AlphaStar**, **robotic manipulation с sparse rewards**, и **offline RL**. Для получения информации о более свежих результатах (2023–2024) можно уточнить запрос, например, по конкретным методам (например, *offline RL*, *transformer-based RL*, *diffusion policies*).

In [ ]:
research("Что нового в computer vision и распознавании объектов?")

In [ ]:
research("Найди статьи про нейросетевое сжатие моделей (model compression)")

---

## Заключение

В этом ноутбуке мы создали **агента-исследователя**, который умеет:

1. **Искать статьи** на arXiv по ключевым словам
2. **Получать детали** о конкретных статьях
3. **Анализировать** и **резюмировать** найденную информацию

### Ключевые концепции:

- **Function Calling** — механизм вызова внешних API через LLM
- **Pydantic-модели** — удобный способ описания функций с валидацией
- **Класс Agent** — универсальная обёртка для обработки вызовов

### Практическое применение:

- 📚 **Литературный обзор** — быстрый поиск релевантных публикаций
- 🔍 **Мониторинг области** — отслеживание новых статей по теме
- 📝 **Подготовка к исследованию** — анализ существующих работ

### Возможные улучшения:

- Добавить сохранение результатов в файл
- Реализовать фильтрацию по дате публикации
- Добавить функцию скачивания PDF
- Создать функцию сравнения нескольких статей